In [ ]:
!pip install konlpy

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 19.4 MB 545 kB/s 
     |████████████████████████████████| 453 kB 84.5 MB/s 


In [ ]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
from urllib import parse
from tqdm.notebook import tqdm

from konlpy.tag import Okt # komoran, hannanum, kkma, mecab

import os

import numpy as np

from datetime import datetime
import json
import re

from collections import Counter
import csv 
import time

# 크롤링

# 아스키코드 변환 함수

In [ ]:
# 한글 검색어(key)를 아스키코드로 변환하는 함수
def ascii(key):
  test = []
  for i in tqdm(range(len(key))):                                       
    ts = key[i].split()
    if len(ts) == 1 :                                                   # 띄어쓰기가 없는 검색어(key)를 아키코드로 변환
      ts_par = parse.quote(ts[0])
      ascii_list.append(ts_par)
    elif len(ts) > 1:                                                   # 띄어쓰기가 된 검색어(key)를 아키코드로 변환 할때 아키코드+아키코드 로 만들기 위한 코드
      for j in ts:
        test.append(parse.quote(j))
      if len(test) == 2:                                                # 띄어쓰기가 1개일 때
        t_tag = test[0] + '+' + test[1]
        ascii_list.append(t_tag)
        test.clear()
      elif len(test) == 3:                                              # 띄어쓰기가 2개 일때
        t_tag = test[0] + '+' + test[1] + '+' + test[2]                
        ascii_list.append(t_tag)
        test.clear()
      elif len(test) == 4:                                              # 띄어쓰기가 3개 일때
        t_tag = test[0] + '+' + test[1] + '+' + test[2] + '+' + test[3]   
        ascii_list.append(t_tag)
        test.clear()
#=============================================================================

In [ ]:
# 언론사별 키워드로 본문등 크롤링
def news_trend(press):
# 한개의 언론사로 10개의 키워드를 차례대로 검색해서 나오는 제목 , 제목의 href, 날짜를 2022년으로 조건을 걸어서 저장
  for key in tqdm(press.keys()):   # 언론사 딕셔너리의 키값들
    temp_tags = []  # 중복 체크를 위한 임시 리스트
    
    for ascii_key in tqdm(ascii_list):     # 아스키코드로 변환된 키워드
      if press[key] == '디지털타임스':    # 특정 언론사에 대한 페이지 조건
        num = 0
      else:
        num = 1
      break_year = []
      for number in range(100):       # 페이지 스탑을 위한 포문
        url = press[key][0].format(num, ascii_key)          # 언론사 딕셔너리에 저장되어있는 0번째 url을 가지고 와서 num과 joy를 format함
        try:
          time.sleep(1)
          resp = requests.get(url, headers=headers)     # 저장된 url을 get으로 가져와서 resp에 저장
        except:
          time.sleep(3)
          resp = requests.get(url, headers=headers)     # 저장된 url을 get으로 가져와서 resp에 저장
        soup = BeautifulSoup(resp.content, 'lxml')    
        a_tit_tags = soup.select(press[key][1]) # 제목의 a태그
        a_day_tags = soup.select(press[key][2]) # 날짜 태그
        # 날짜에 따른 제목 a태그 가져오는 테스트

        for i in range(len(a_day_tags)):
          day = re.sub("[^0-9]", "", a_day_tags[i].text)

          if day[:4] == '2022' or day[:4] == '2021': # tag[i].text[15:19] 
            temp_tags.append(a_tit_tags[i]['href'])

          elif day[:4] == '2020':
            break_year.append(day[:4])
            n = 1
            break

        if '2020' in break_year:
          break
        else:
          if press[key] == '디지털타임스':
            num += 20
          else :
            num += 1

      # 2. 가져온 href가 중복인지 거르고 
    overlap = list(set(temp_tags))
    

    for l in tqdm(range(len(overlap))):
      
      try:
        time.sleep(1)
        if overlap[l][0] == 'h':
          href_url = overlap[l]
        else:
          href_url = press[key][3]+overlap[l]
      except:
        time.sleep(3)
        if overlap[l][0] == 'h':
          href_url = overlap[l]
        else:
          href_url = press[key][3]+overlap[l]
      
      
      href_resp = requests.get(href_url, headers=headers)
      href_soup = BeautifulSoup(href_resp.content, 'lxml')

      tit_tags = href_soup.select(press[key][4]) # 본문의 제목 태그
      day_tags = href_soup.select(press[key][5]) # 본문의 날짜 태그
      bon_tags = href_soup.select(press[key][6]) # 본문의 본문 태그

      
                  
      for tit, day in zip(tit_tags, day_tags):
        total_press.append(key)
        total_tit.append(tit.text)
        total_day.append(re.sub("[^0-9]", "", day.text)[:8])
        
        p_gather =[]      # 리스트 안에 리스트로 만들면서 0번째로 인식시키기 위함
        for bon in bon_tags:   # p태가 여러개이므로 제목 한개에 본문 한개로 인식시키기 위함
          if bon.text != '':
            p_gather.append(bon.text)

        total_bon.append(p_gather)

In [ ]:

# 검색할 한글 키워드 리스트
key_list = ['캐논코리아', '캐논 카메라','소니 카메라','풀프레임', '미러리스', '카메라', '디지털카메라', '소니코리아', 'DSLR', '삼성 카메라'] # 캐논 소니 라이카 
#key_list = ['소니코리아'] # 캐논 소니 라이카 

# 아스키코드 저장할 리스트
ascii_list = []

# key_list를 아스키코드로 변환후 arkey_list에 append하는 함수
ascii_def = ascii(key_list)



# 언론사 딕셔너리
#   '언론사 이름' : 0. 검색했을때 나오는 url, 1. 검색했을때 나오는 제목의 a태그 , 2. 검색했을때 나오는 날짜 태그 , 
#                   3. 언론사 기본 url , 4. 본문의 제목 , 5. 본문의 날짜, 6.본문의 본문
press = {
    'aving':['https://kr.aving.net/news/articleList.html?page={0}&total=839&box_idxno=&sc_area=A&view_type=sm&sc_word={1}', 'div.view-cont h4.titles a', 'div.view-cont span.byline', 
             'http://kr.aving.net', 'h3.heading', 'div.view-info ul li', 'div.article-body p'],
    'zdnet':['https://search.zdnet.co.kr/?kwd={1}&area=&pageno={0}&term=3', 'div.assetText a', 'p.byline span', 
             'https://zdnet.co.kr', 'div.news_head h1', 'p.meta span', 'div.view_cont p'],
    '전자신문' : ['https://search.etnews.com/etnews/search.html?category=CATEGORY1&kwd={1}&pageNum={0}&pageSize=10&reSrchFlag=false&sort=1&startDate=\
        &endDate=&sitegubun=&jisikgubun=&preKwd%5B0%5D={1}', 'dl.clearfix dt a', 'div.list_search dd.date span',
                  'https://www.etnews.com', 'div.article_title h2', 'div.article_header_sub time.date', 'div.article_txt p'],
    '뉴시스' : ['https://newsis.com/search/?val={1}&sort=acc&jo=all_jogun&bun=all_bun&sdate=&term=allday&edate=&s_yn=Y&catg=1&t=1&page={0}&', 'ul.articleList2 p.tit a', 'div.txtCont p.time',
                'https://newsis.com', 'div.top p.tit.title_area', 'div.left p.txt', 'div.viewer article'],
    'IT동아' : ['https://it.donga.com/search/?page={0}&query={1}', 'li.media.my-4 a', 'div.mb-2.row div.col-sm-8.text-right time',
                  'https://it.donga.com/', 'header.mb-3.pb-0 h1', 'div.row.py-2 time', 'article.mb-3 p'],
    'IT조선' : ['https://it.chosun.com/svc/list_in/search.html?query={1}&sort=regdate&pn={0}', 'div.thumb_news div.txt_wrap > a', 'span.date',
              'http://it.chosun.com', 'div.news_title_text h1', 'div.news_date', 'div#news_body_id div.par'],
    '디지털타임스':['http://search2.dt.co.kr/newsList.jsp?section=news&txt={1}&f={0}&psection=1&sort=DATE/DESC','ul.s_tit a', 'div.dates',
                     'http://www.dt.co.kr','h1.top_title', 'li.lasttime', 'div.art_txt'],
    '헤럴드경제':['http://biz.heraldcorp.com/search/index.php?q={1}&sort=1&np={0}', 'div.list a', 'div.l_date',
                      'http://biz.heraldcorp.com', 'li.article_title.ellipsis2', 'li.article_date', 'div.article_view p'],
    '디지털데일리':['https://www.ddaily.co.kr/news/search_result.html?search={1}&search_mode=&hash=&start_date=&end_date=&page={0}', 'a.gray143c', 'span.font11blue2',
                    'https://www.ddaily.co.kr/news/', 'div.hbox h2', 'p.arvdate', 'div.smartOutput'],
    '스마트PC사랑' : ['https://www.ilovepc.co.kr/news/articleList.html?page={0}&total=251&box_idxno=&sc_area=A&view_type=sm&sc_word={1}', 'div.list-titles a', 'div.list-dated',
                      'https://www.ilovepc.co.kr', 'div.article-head-title', 'ul.no-bullet.auto-marbtm-0.line-height-6', 'div#article-view-content-div p']
    
    
    
    
}

headers = {
    'User-Agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/103.0.0.0 Safari/537.36'
}


total_press, total_tit, total_day, total_bon = [], [], [], []

trend = news_trend(press)


dict_news = {
    '언론사':total_press,
    '제목':total_tit,
    '날짜':total_day,
    '본문':total_bon
} 
#print(len(t_j))
# 딕셔너리 = 언론사 제목 본론 날짜

df_news = pd.DataFrame(dict_news)
df_news.to_csv("total_news.csv")

# 현재 문제점
# n_b 라는 딕셔너리만들고 나서 value값의 갯수가 맞지 않아 DataFrame가 만들어지지 않음
# 추가로 href만 중복처리를 했는데 다른것들도 해야 맞는것이 아닌지
# 이러면 언론사 딕셔너리에서 뺏었던 두개를 다시 넣어야 되는것이 아닌지....
# 슬립함수 사용시 그나마 좀 괜찮다?
# 슬립 함수를 썻음에도 불구하고 10개의 키워드로 10개의 언론사는 안된다.
# 10개의 키워드로 2개의 언론사

# 전처리

In [ ]:
def preprocessing(trend):
    okt = Okt()
    
    # f = open('nsmc_stopwords.txt')
    # stop_words = f.read().split()
    
    # 1. 한글 및 공백을 제외한 문자 모두 제거.
    trend_text = re.sub("[^가-힣a-zA-Z\\s]", "", trend)
    
    # 2. okt 객체를 활용해서 형태소 토큰화 + 품사 태깅
    word_trend = okt.pos(trend_text, stem=True) # stem = 어감추출 - 통일화?

    # 노이즈 & 불용어 제거
    word_trend = [(token,pos) for token, pos in word_trend if token not in stopwords and len(token) > 1 and len(token) <= 10] # if not token과 if token not 의 차이는 문법의 차이이다.

    # 명사, 동사, 형용사 추출
    word_trend = [token for token, pos in word_trend if pos in ['Noun','Alpha','Josa','Suffix','Adjective']]#, 'Verb', 'Adjective'

    #preprocessed = ' '.join(word_trend)

    return word_trend

    #return word_trend

In [ ]:
stopwords = []

with open("트랜드_불용어사전2.txt", 'rt') as f:
  stopwords = f.read().split()
stopwords = list( set(stopwords) )
#stopwords

In [ ]:
df_total_news = pd.read_csv('total_news.csv')

df_total_news.astype({'날짜':'int64'}) # 날짜 컬럼의 데이터를 str에서 int로 변환
df_total_news_2022 = df_total_news[df_total_news['날짜'] > 20220000 ] # 2022년 이상인 데이터를 저장
df_total_news_2022 = df_total_news_2022.sort_values(by=['날짜']) # 날짜순으로 정령
df_total_news_2022 = df_total_news_2022.reset_index(drop=True) # 필요없는 인덱스 컬럼 제거
df_total_news_2022 = df_total_news_2022.drop(['Unnamed: 0'], axis=1) # 필요없는 인덱스 컬럼 제거

df_total_news_2021 = df_total_news[df_total_news['날짜'] < 20220000 ]
df_total_news_2021 = df_total_news_2021.sort_values(by=['날짜'])
df_total_news_2021 = df_total_news_2021.reset_index(drop=True)
df_total_news_2021 = df_total_news_2021.drop(['Unnamed: 0'], axis=1)
#df_total_news_2021_2 = df_total_news[202106000 < df_total_news['날짜'] < 20220000 ]

df_total_news_2022.to_csv("total_news_2022.csv")
df_total_news_2021.to_csv("total_news_2021.csv")
#df_total_news_2021_2.to_csv("total_news_2021_2.csv")

# news_vs = df_total_news['aving' == df_total_news['언론사']]
# news_vs
#total_news[['본문']].info()

In [ ]:
# df_total_news_2022 = pd.read_csv("total_news_2022.csv")
df_total_news_2021 = pd.read_csv("total_news_2021.csv")

df_total_news_2021 = df_total_news_2021[:1000]

# display(df_total_news_2022)
display(df_total_news_2021)

,Unnamed: 0,언론사,제목,날짜,본문
0,0,IT조선,삼성전자 갤럭시S21 티저 광고 공개,20210101,['삼성전자는 2020년 12월 31일(이하 현지시각) 스마트폰 신제품 갤럭시S21...
1,1,뉴시스,"국내연구진, 머리카락보다 100배 얇은 초박막렌즈 개발 성공",20210101,"['\n\n적외선 내시경, CCTV, 야간투시경 등 분야에 응용 가능\n\n\n[서..."
2,2,뉴시스,"갤S21, 역대급 카메라 성능?…유출된 '언팩 초청장' 보니",20210101,['\n\n언팩 2주 앞두고 삼성 ‘언팩’ 초청장 영상 일부 유출\r\n줌으로 순간...
3,3,디지털타임스,스마트폰 렌즈 두께 확 줄여줄 ‘초박막렌즈’ 나와,20210101,['\n\n\n노준석 포항공대 교수 연구팀이 기존 굴절렌즈보다 두께가 1만배 얇은 ...
4,4,전자신문,출시 앞둔 삼성 '갤럭시S21' 총정리…바이올렛 색상으로 '깔맞춤'!,20210101,['삼성전자 차세대 전략 스마트폰 ‘갤럭시S21’ 시리즈 공개가 2주 앞으로 다가왔...
...,...,...,...,...,...
995,995,헤럴드경제,에쓰오일 모든 사업장 디지털 전환,20210310,"['에쓰오일이 모든 사업장을 디지털로 전환한다.', '에쓰오일은 최근 디지털 전환 ..."
996,996,스마트PC사랑,"니콘, 플래그십 풀프레임 미러리스 카메라 'Z 9' 개발 발표",20210310,"[""[smartPC사랑=이철호 기자]\xa0광학기기 전문기업 니콘은 Z 마운트를 채..."
997,997,헤럴드경제,“김샜다?” 삼성 17일 공개 역대급 ‘가성비폰’ 다 유출! [IT선빵!],20210310,"['[헤럴드경제=박혜림 기자] “이번에도 갤럭시 언팩, 다 유출됐다!”', '삼성전..."
998,998,aving,"[K-chemicals 2021] 위드닉스, 주름개선 및 피부 재생을 촉진하는 'S...",20210310,"[""위드닉스(대표 노만응)은 오는 3월 22일부터 26일까지 '2021 화학 분야 ..."


In [ ]:
from collections import Counter

stopwords_str = '\n'.join(stopwords)

#news = newsis.bon # 기사 본분만 news에 저장

# 22년 전처리
# pre_total_2022 = []
# df_total_news_2022['article_preprocessed'] = ''
# for y in tqdm(range(len(df_total_news_2022['본문']))):
#     word_trend = preprocessing(df_total_news_2022['본문'][y])
#     pre_total_2022 += word_trend
#     df_total_news_2022['article_preprocessed'].iloc[y] = ' '.join(word_trend)

# 21년 전처리
pre_total_2021 = []
df_total_news_2021['article_preprocessed'] = ''
for z in tqdm(range(len(df_total_news_2021['본문']))):
    word_trend = preprocessing(df_total_news_2021['본문'][z])
    pre_total_2021 += word_trend
    df_total_news_2021['article_preprocessed'].iloc[z] = ' '.join(word_trend)

# 22년 counter
# word_con_2022 = Counter(pre_total_2022)
# 21년 counter
word_con_2021 = Counter(pre_total_2021)

from tqdm import tqdm
tqdm.pandas() 

#df_total_news['article_preprocessed']= df_total_news['본문'].progress_apply(lambda x: preprocessing(' '.join(x)))
#pre_total
# all_of_count_2022 = sorted(word_con_2022.items(), key=lambda x: x[1], reverse=True)
all_of_count_2021 = sorted(word_con_2021.items(), key=lambda x: x[1], reverse=True)

# word_ranking_2022 = dict(all_of_count_2022[:20])
word_ranking_2021 = dict(all_of_count_2021[:20])
#word_ranking

100%|██████████| 1000/1000 [01:35<00:00, 10.52it/s]


In [ ]:
# all_of_count_2022 = sorted(word_con_2022.items(), key=lambda x: x[1], reverse=True)
all_of_count_2021 = sorted(word_con_2021.items(), key=lambda x: x[1], reverse=True)

# word_ranking_2022 = dict(all_of_count_2022[:20])
word_ranking_2021 = dict(all_of_count_2021[:20])


# 히스토그램 그리기

In [ ]:
# 그래프에서 한글표현을 위해 폰트를 설치합니다.
!apt-get update -qq
!apt-get install fonts-nanum* -qq

!pip install chart_studio
# pandas에서 바로 plotly로 그래프를 제작해주는 라이브러리 
!pip install cufflinks

#pandas profiling
!pip install -U pandas-profiling

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
!pip install pyyaml==5.4.1

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


## 여기서부터

In [ ]:
import matplotlib.pyplot as plt  # 그래프 그리는 용도
import matplotlib.font_manager as fm  # 폰트 관련 용도
import matplotlib as mpl

path = '/usr/share/fonts/truetype/nanum/NanumGothicEco.ttf'  # 설치된 나눔글꼴중 원하는 녀석의 전체 경로를 가져오자
font_name = fm.FontProperties(fname=path, size=10).get_name()
print(font_name)
# 폰트 지정
plt.rc('font', family=font_name)

#마이너스 깨짐현상
mpl.rc('axes', unicode_minus=False)

fm._rebuild()

NanumGothic Eco


In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from IPython.display import display

import pandas as pd
import numpy as np
import chart_studio.plotly as py
import cufflinks as cf
import seaborn as sns
import plotly.express as px



%matplotlib inline

# Make Plotly work in your Jupyter Notebook
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
init_notebook_mode(connected=True)
# Use Plotly locally
cf.go_offline()

# default renderer 세팅을 바꿔주기 
# https://plotly.com/python/renderers/#setting-the-default-renderer

import plotly.io as pio
pio.renderers.default = 'colab'

In [ ]:
list_of_maker = ['word_ranking_2022', 'word_ranking_2021']

list_of_df = []
list_of_dic = []

noun_count_list = [word_ranking_2022, word_ranking_2021]

for i, maker in enumerate( list_of_maker ):
  tmp_dic = {
    'Word' : [],
    'Count' : []
  }

  for j, k in noun_count_list[i].items():

    tmp_dic['Word'].append( j )
    tmp_dic['Count'].append( k )

  tmp_df = pd.DataFrame( tmp_dic )
  list_of_dic.append(tmp_dic)
  list_of_df.append(tmp_df)
  tmp_df.to_csv( 'df_keyword_n_count_of_{0}' .format( maker ) )

for i, j in enumerate(list_of_df): #데이터프레임 출력
  print(list_of_maker[i])
  display(j)
  print()

word_ranking_2022


,Word,Count
0,기술,10390
1,서비스,9805
2,제품,7022
3,제공,6966
4,사업,6715
5,시장,6409
6,기업,6233
7,삼성,6101
8,출시,5417
9,에는,5407



word_ranking_2021


,Word,Count
0,기술,1974
1,제품,1565
2,기능,1417
3,탑재,1296
4,지원,1155
5,서비스,1148
6,시장,1128
7,시리즈,1078
8,사용,1058
9,적용,1040


In [ ]:
for i in list_of_df:
  a = px.histogram( i, x='Word', y='Count', color='Word' )
  a.show()

# 토크나이저


In [ ]:
def my_tokenizer(text):
  return text.split()

In [ ]:
stopwords = []

with open("트랜드_불용어사전2.txt", 'rt') as f:
  stopwords = f.read().split()
stopwords = list( set(stopwords) )
#stopwords

In [ ]:
# sklearn을 통해서 TF-IDF 행렬을 만들어 줍니다. 
from sklearn.feature_extraction.text import TfidfVectorizer

tfidf_vect = TfidfVectorizer(tokenizer=preprocessing, ngram_range=(2, 2))
tfidf = tfidf_vect.fit_transform(raw_documents = df_total_news_2021['article_preprocessed'].iloc[:]) # 기사 개수는 1000개로 조절하겠습니다

tfidf

<1000x120192 sparse matrix of type '<class 'numpy.float64'>'
	with 170121 stored elements in Compressed Sparse Row format>

In [ ]:
from sklearn.decomposition import LatentDirichletAllocation

lda = LatentDirichletAllocation(n_components =5, 
                                max_iter = 20,
                                learning_method='online',
                                random_state=100)
lda_output = lda.fit_transform(tfidf)

In [ ]:
print('lda components:\n',lda.components_) 
print('shape:', lda.components_.shape) 

In [ ]:
terms = tfidf_vect.get_feature_names() # 단어 집합

def get_topics(components, feature_names, n=7):
    for idx, topic in enumerate(components):
        print("Topic %d:" % (idx+1), [(feature_names[i], topic[i].round(2)) for i in topic.argsort()[:-n - 1:-1]])
get_topics(lda.components_,terms)

Topic 1: [('인공 지능', 1.43), ('색상 가격', 1.41), ('보상 프로그램', 1.33), ('중고 보상', 1.18), ('혁신 제품', 1.12), ('서피스 허브', 1.06), ('ram 메모리', 1.05)]
Topic 2: [('인공 지능', 2.41), ('지능 ai', 1.34), ('ai 기술', 1.25), ('생산 능력', 1.22), ('블루 스퀘어', 1.13), ('콘텐츠 제작', 1.12), ('처리 속도', 1.09)]
Topic 3: [('화소 광각', 4.7), ('화소 화소', 3.82), ('천만 화소', 3.59), ('광각 화소', 3.45), ('용량 배터리', 2.91), ('화소 접사', 2.25), ('화소 망원', 2.01)]
Topic 4: [('이미지 센서', 5.84), ('기술 적용', 2.47), ('센서 시프트', 2.24), ('선명하다 사진', 1.89), ('사진 촬영', 1.89), ('촬영 가능하다', 1.76), ('픽셀 기술', 1.67)]
Topic 5: [('풀프레임 미러리스', 3.03), ('소니코리아 알파', 2.17), ('ef 렌즈', 1.73), ('캐논 플렉스', 1.72), ('대상 제품', 1.49), ('인공 지능', 1.44), ('알파 알파', 1.27)]


/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning:

Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.



================================================================================================================================================================여기 밑에서 부터 테스트 코드

In [ ]:
# df_total_news_2022 = df_total_news_2022.reset_index(drop=True)
# df_total_news_2022 = df_total_news_2022.drop(['Unnamed: 0'], axis=1)
df_total_news_2022.sort_values(by=['날짜'])


In [ ]:
# '매일경제':['https://find.mk.co.kr/new/search.php?pageNum={0}&cat=&cat1=&media_eco=&pageSize=10&sub=all&dispFlag=OFF&page=news&s_kwd={1}\
#     &s_page=news&go_page=&ord=1&ord1=1&ord2=0&s_keyword=%C4%B3%B3%ED+%C4%AB%B8%DE%B6%F3&s_i_keyword=%C4%B3%B3%ED+%C4%AB%B8%DE%B6%F3&s_author=\
#     &y1=2019&m1=07&d1=29&y2=2022&m2=07&d2=29&ord=1&area=ttbd', 'div.sub_list span.art_tit a', 'div.sub_list span.art_time',
#                 'https://www.mk.co.kr', 'div.news_title_text h1.top_title', 'div.news_title_author ul li.lasttime', "div#Content td[align='left']"]
# 매일경제는 일단 보류 아이프레임이라 

# '경향신문' : ['https://search.khan.co.kr/search.html?stb=khan&q={1}&pg={0}&sort=1', 'dl.phArtc dt a', 'dl.phArtc dt span.date',
    #               'https://www.khan.co.kr', 'div.subject h1#article_title', 'div.function_wrap div.art_info div.byline', 'div.art_body p']
# 경향신문은 어째선지 안가져와짐 
특정 언론사는 iframe과 JavaScript가 아님에도 불구하고 데이터를 못가져오는 현상으로 인해서 제외.

# '연합뉴스' : ['https://www.yna.co.kr/search/index?query={1}&ctype=A&page_no={0}', 'div.cts_atclst ul li a', 'div.cts_atclst span.pbdt',
    #           'https:', 'div.content03 h1.tit', 'div.content03 p.update-time', 'div.content01.scroll-article-zone01 div div article.story-news.article p'],
# 연합뉴스는 json임

In [ ]:
from itertools import islice

# 다시 딕셔너리로 변환하고 순서를 정리하는 방향
count_tag = dict(islice(count.items(), 10))
count_tag

# df_petition['votes'].iplot(kind='hist') 그래프 그리기
# df_petition['category'].value_counts() #빈도수?

{'되다': 103,
 '렌즈': 158,
 '영상': 92,
 '있다': 245,
 '제품': 71,
 '지원': 64,
 '촬영': 167,
 '캐논': 225,
 '코리아': 99,
 '하다': 575}

In [ ]:
try:
  time.sleep(1)
except:
  time.sleep(3)

In [ ]:
tsv = [1,2,3,4,5,6]
svt = ['가','나','다','라','마','바']
for i in range(len(tsv)):
  n = 1
  for j in range(len(svt)):
    if svt[n] == '마':
      print(i)
      break
    n+=1


0
1
2
3
4
5


In [ ]:
a = '12가나 abC D'


b = re.sub("[^가-힣a-zA-Z]", "", a)
b

'가나abCD'

In [ ]:
df_total_news = pd.read_csv('total_news.csv')
okt = Okt()
trend_text = re.sub("[^가-힣a-zA-Z\\s]", "", df_total_news['본문'][0])
word_trend = okt.pos(trend_text, stem=True)
word_trend

In [ ]:
a=[]
for i in range(len(tag)):
  a.append(re.sub("[^0-9.: ]", "", tag[i].text))
  # if tag[i].text[15:19] == '2022':
  #   tage.append(t_tag[i]['href'])
        
a[0]

'2021.04.01 16:43'

In [ ]:
t = [1,2,3]
s = ['a','b','c']
d = ['가','나','다']
td = {
    '제목':t,
    '날짜':s,
    '본문':[d,d,d]
}
tb = pd.DataFrame(td)
tb.to_csv('test.csv')
tb

In [ ]:
from pandas.io.formats.format import DataFrameRenderer
import csv 

# tb_t = open('test.csv', 'r')

df = pd.read_csv('df_wep.csv')
df['본문']

0     ['캐논코리아 주식회사(대표이사 최세환)가 최근 카메라 솔루션과 함께 프린팅 솔루션...
1     ['캐논코리아 주식회사(대표이사 최세환)가 네트워크 카메라 영상에서 움직이는 물체를...
2     ["캐논코리아 주식회사(대표이사 최세환)가 광각 단초점 렌즈 'RF24mm F1.8...
3     ["캐논코리아 주식회사(대표이사 최세환)가 초경량 풀프레임 미러리스 카메라 'EOS...
4     ["캐논코리아 주식회사(대표이사 최세환)가 디즈니 토이 스토리와 컬래버레이션해 미니...
5     ["글로벌 미디어 채널 에이빙 뉴스가 지난 6월 29일(수) 서울 코엑스(COEX)...
6     ["캐논코리아 주식회사(대표이사 최세환)가 29일 삼성동 코엑스(COEX) 전시장 ...
7     ["캐논코리아 주식회사(대표이사 최세환)가 자연 속 다양한 야생 조류의 모습을 담은...
8     ["캐논코리아 주식회사(대표이사 최세환)가 EOS R 시리즈 최초의 APS-C 미러...
9     ["캐논코리아 주식회사(대표이사 최세환)가 오는 6월 15~18일 4일간 서울 강남...
10    ["캐논코리아 주식회사(대표이사 최세환)가 'KLPGA 2022 롯데오픈' 골프대회...
11    ["캐논코리아(대표이사 최세환)가 'TIPA 어워드 2022(TIPA Award 2...
12    ["캐논코리아(대표이사 최세환)가 EOS R 시리즈 최초로 APS-C 타입 CMOS...
13    ["캐논코리아 주식회사(대표이사 최세환)가 새로운 비즈니스 잉크젯 프린터 'GX50...
14    ["캐논코리아 주식회사(대표이사 최세환)가 AI 카메라 '파워샷 픽'의 정식 판매를...
15    ['캐논코리아 주식회사(대표이사 최세환)가 5월을 맞아 온·오프라인 강좌를 확대 운...
16    ["캐논코리아 주식회사(대표이사 최세환)가 강원도관광재단(대표이사 강옥희)과 함께 ...
17    ["캐논코리아 주식회사(대표이사 최세환)가 '2022 iF(Internati

In [ ]:
tf = df.본문
trend_text=[]
for i in range(len(tf)):
  trend_text = re.sub("[^가-힣\\s]", "", tf[i])
  word_trend = okt.pos(trend_text, stem=True) # stem = 어감추출 - 통일화?
  print(word_trend)


[('가다', 'Verb'), ('나', 'Noun'), ('다', 'Adverb')]
[('가다', 'Verb'), ('나', 'Noun'), ('다', 'Adverb')]
[('가다', 'Verb'), ('나', 'Noun'), ('다', 'Adverb')]


In [ ]:
j = 1
t = ['도','레','미','파']
for i in range(10):
  
  if t[i] == '미':
    print(j)
    j = 1
    break;
  j = j+1
print(i,j)

3
2 1


In [ ]:
t = [1,2,3]
s = 1
for i in range(len(t)):
  #print(f'{i} 번째 숫자입니다.{s}')
  u = f'{i}번째 숫자 {s}'
  print(u)
  s += 1
print(type(u))

0번째 숫자 1
1번째 숫자 2
2번째 숫자 3
<class 'str'>


In [ ]:
ts = {
    'avi':['{0}번째 숫자 {1}'],
    'tvi':['{1}번째 숫자 {0}']
}

t = [1,2,3]
s = 1
for z in range(len(t)):
  #print(f'{i} 번째 숫자입니다.{s}')
  # print(ts['avi'][0])
  #u = ts['avi'][0].format(z,s)
  us = ts['tvi'][0].format(z,s)
  #print(u)
  print(us)
  s += 1

1번째 숫자 0
2번째 숫자 1
3번째 숫자 2


In [ ]:
t = {
    1 : [1,2,3],
    2 : [4,5,6]
}
for kyes in t.keys():
  print(t[kyes][0])

1
4


In [ ]:
headers = {
    'User-Agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/103.0.0.0 Safari/537.36'
}
# '전자신문' : ['https://search.etnews.com/etnews/search.html?category=CATEGORY1&kwd={1}&pageNum={0}&pageSize=10&reSrchFlag=false&sort=1&startDate=\
#         &endDate=&sitegubun=&jisikgubun=&preKwd%5B0%5D={1}', 'dl.clearfix dt a', 'div.list_search dd.date span',
#                   'https://www.etnews.com', 'div.article_title h2', 'div.article_header_sub time.date', 'div.article_txt p'],

# day = re.sub("[^0-9]", "", a_day_tags[i].text)

#           if day[:4] == '2021': # tag[i].text[15:19] 
            
#             temp_tags.append(a_tit_tags[i]['href'])
#           elif day[:4] == '2022':
#             break
n =1
da = []
days = []
for w in range(100):
  
  url = 'https://search.etnews.com/etnews/search.html?category=CATEGORY1&kwd=%EC%BA%90%EB%85%BC%EC%BD%94%EB%A6%AC%EC%95%84&pageNum={}&pageSize=10&reSrchFlag=false&sort=1&startDate=&endDate=&sitegubun=&jisikgubun=&preKwd%5B0%5D=%EC%BA%90%EB%85%BC%EC%BD%94%EB%A6%AC%EC%95%84'.format(w+1)
  resp = requests.get(url, headers=headers)
  soup = BeautifulSoup(resp.content, 'lxml')
  tag = soup.select('div.list_search dd.date span')
  tags = soup.select('dl.clearfix dt a')
  for i in range(len(tag)):
    day = re.sub("[^0-9]", "", tag[i].text)
    if day[:4] == '2022' or day[:4] == '2021': # tag[i].text[15:19] 
      da.append(tags[i].text)
      print('a',i)
    elif day[:4] == '2020':
      dayss = day[:4]
      print('b',i)
      break

  if dayss == '2020':
    break
    
    # if day[:4] == '2022':
    #   if i == len(tag)-1:
    #     n +=1
    #     break
    #   continue
    # elif  day[:4] == '2021':
    #   da.append(day[:4])
    #   if i == len(tag)-1:
    #     n +=1
    #     break
    # elif day[:4] == '2020':
    #   n =1
    #   break

a 0
a 1
a 2
a 3
a 4
a 5
a 6
a 7
a 8
a 9


NameError: ignored

In [ ]:
len(da)

43

In [ ]:
headers = {
    'user-agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/103.0.0.0 Safari/537.36'
}

test_tag = []
turl = 'https://search.khan.co.kr/search.html?stb=khan&q=%EC%BA%90%EB%85%BC+%EC%B9%B4%EB%A9%94%EB%9D%BC&pg=2&sort=1'

tresp = requests.get(turl, headers=headers)
print(tresp)
soup = BeautifulSoup(tresp.content, 'lxml')
tag = soup.select('dl.phArtc dt a')
#tag = soup.select('div.list-dated')
for io in tag:
  print(io['href'])

In [ ]:
headers = {
    'user-agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/103.0.0.0 Safari/537.36'
}


turl = 'http://news.khan.co.kr/kh_news/khan_art_view.html?artid=201805130931021&code=940202'

tresp = requests.get(turl, headers=headers)
soup = BeautifulSoup(tresp.content, 'lxml')
#tag = soup.select('div.article-head-title')
#tag = soup.select('ul.no-bullet.auto-marbtm-0.line-height-6')
# tag = soup.select('div#container h1.headline')
# tag
soup

In [ ]:
import json

headers = {
    'refere':'https://www.yna.co.kr/',
    'User-Agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/103.0.0.0 Safari/537.36'
}

test_tag = []
url = 'https://ars.yna.co.kr/api/v2/search.asis?callback=Search.SearchPreCallback&query=%EC%BA%90%EB%85%BC%EC%BD%94%EB%A6%AC%EC%95%84&page_no=2&ctype=A&page_size=10&channel=basic_kr'
resp = requests.get(url, headers=headers)


soup = BeautifulSoup(resp.content, 'lxml')

soup




<html><body><p>Search.SearchPreCallback({"CONVERT_TIME":0,"ctype":"A","LOAD":1,"_V":"NEW","channel":"basic_kr","error":"Invalid Parameter [ua]","KR_ARTICLE":{"result":[],"query":"캐논코리아","count":0,"page_no":2,"totalCount":0,"collName":"KR_ARTICLE","page_size":10},"URL":""})
</p></body></html>

In [ ]:
data_tag = []
headers = {
    'User-Agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/103.0.0.0 Safari/537.36'
}
for vo in test_tag:
  url = vo
  resp = requests.get(url, headers=headers)
  soup = BeautifulSoup(resp.content, 'lxml')
  tag = soup.select("div#Content td[align='left']")

  test = []
  for tt in tag:
    test.append(tt.text)
  data_tag.append(test)

data_tag[0]

In [ ]:
data_tag[1]

[]

In [ ]:
url = 'http://kr.aving.net/news/articleList.html?sc_area=A&view_type=sm&sc_word=%EC%BA%90%EB%85%BC+%EC%B9%B4%EB%A9%94%EB%9D%BC'
resp = requests.get(url)
soup = BeautifulSoup(resp.content, 'lxml')
tag = soup.select('div.view-cont span.byline')
tiving=[]
for ta in tag :
  rs = re.sub("[^0-9]", "", ta.text)[:8]
  tiving.append(rs)
tiving

할 일
#스크롤사용하는 방법 찾기
1,언론사 10개 에서 각 키워드 10개 기사갯수 최소 1000개 추출
2,각 키워드 별 100개의 블로그 본문내용 추출
3,각 키워드 별 유튜브 100개의 제목과 소개글 추출
4,각 키워드 별 인스타그램 인기개시물 100개 최신개시물 100개 추출
5.1~4번까지 DB에 저장 후 도출해야되는 빈도수가 높은 키워드 갯수 20개 순위대로 도출

몽고DB를 활용 저장할 때 글자수 제안 확인해보기

1번을 최소의 갯수로 csv로 만들어보고, 몽고DB로 저장해보기

1차 1,2,3,4번 전체 빈도수가 높은 키워드,
2차 1,2 - 3,4 두 그룹으로 나눠서 보는걸로

검색을 하고 본문내용을 DB

In [ ]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
from urllib import parse
from tqdm.notebook import tqdm

In [ ]:
# 각 언론사별 키워드를 검색해서 나오는 제목의 href 구하기
def url_taget(a_url):
  for key in a_url:
    for i in range(len(arkey_list)):
      for j in range(1,6):
        url = a_url[key][0].format(j, arkey_list[i])       # a_url에 있던 url을 가져오고 아키코드로 변환 한 arkey_list적용후 url에 저장
        resp = requests.get(url, headers= headers)      
        soup = BeautifulSoup(resp.content, 'lxml')
        a_tag = soup.select(a_url[key][1])              # a_url에 저장되어 있던 태그를 select해서 a_tag에 저장
        b_tag = soup.select(a_url[key][2])
        for l in range(len(b_tag)):
          if b_tag[l].text[15:19] == '2022':
            for te in a_tag:
              if key == 'aving' :                           # 각 언론사의 이름에 맞춰서 href 저장
                t.append(te['href'])
              elif key == 'zdnet':
                s.append(te['href'])
              elif key == '동아일보':
                r.append(te['href'])                        # 언론사 추가시 밑으로 더 추가하면 됨

In [ ]:
# 처음 검색 후 나오는 제목(href)의 중복 제거
def overlap(t,s,r):
  for v,n,w in zip(t,s,r) :           # 위에서 저장한 변수들을 가져오고 
    if v not in x:                    # t를 v에 넣는 동시에 if로 x에 v가 없는지, 없는게 확인 되어서 True면 x에 append
      x.append(v)
    if n not in y:
      y.append(n)
    if w not in z:
      z.append(w)

In [ ]:
# 검색할 한글 키워드 리스트
key_list = ['캐논코리아', '캐논 카메라'] # 캐논 소니 라이카 

# 아키코드 저장할 리스트
arkey_list = []

# 언론사 별 딕셔너리 만들기
# 딕셔너리 변수 이름 = {'언론사이름':['검색할 언론사 사이트', '기사의 제목 a태그', '기사의 날짜 태그']}
a_url = {
    'aving' : ['http://kr.aving.net/news/articleList.html?page={}&total=839&box_idxno=&sc_area=A&view_type=sm&sc_word={}', 'div.view-cont h4.titles a', 'div.view-cont span.byline', 'http://kr.aving.net', 'div.view-cont h4.titles a']
    #'zdnet':['https://search.zdnet.co.kr/?kwd={}', 'div.newsPost div.assetText a', ''],
    #'동아일보':['https://www.donga.com/news/search?check_news=1&more=1&sorting=1&range=1&search_date=&v1=&v2=&query={}', 'div.searchList p.tit a', '']
}

# 페이징 처리는 3페이지 까지.

# 각 언론사별 본문 딕셔너리
b_url = {
    'aving' : ['http://kr.aving.net', 'div.view-cont h4.titles a'],
    'zdnet':['https://search.zdnet.co.kr', 'div.newsPost div.assetText a'],
    '동아일보':['https://www.donga.com', 'div.searchList p.tit a']
}



# 언론사별 get해도 안들어가지는 경우가 있어서 headers를 추가
headers = {
    'User-Agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/103.0.0.0 Safari/537.36'
}

# 임시 저장 list
t = []
s = []
r = []
x = []
y = []
z = []

# 처음 검색 후 나오는 제목(href) 가져오는 함수
url_tag1 = url_taget(a_url)

# 가져온 href의 중복 제거 함수
overlap_tag1 = overlap(t,s,r)

print(arkey_list)
print('aving의 전체 기사 갯수:{}'.format(len(t)))
print(len(s))
print(len(r))
print(len(x))
print(len(y))
print(len(z))


# 일단 1개의 키워드와 1개의 언론사로 본문 내용 가져오고 그걸 csv로 만들고, 최빈도수를 도출
# 마찬가지로 블로그, 유튜브, 인스타 도 마찬가지로 1개로만 작업해보기
# 여러개 들어갈 수 있도록 살 붙이기

# https://kr.aving.net/news/articleList.html?page={}&total=900&box_idxno=&sc_area=A&view_type=sm&sc_word={}
# 1번 부터 10번까지의 페이지 중에서 2022년으로 한번 거르고
# 걸러진 제목의 href를 저장하고
# 
# 가져온 href로 들어가서 제목, 날짜 본문 내용 가지고 온 뒤 데이터베이스로 적용하고 csv파일로 저장

# 날짜와 페이지관련 수정요망

# 현재 상태
# 키워드는 정해진걸로 list저장(리스트에 키워드 추가해도 가능하게끔 해놨고, 추가로 input으로 입력하는것도 가능해보이나 현재로선 정해진 갯수로 하는것이 좋아보임)
# 언론사,언론사의 검색했을당시의 url 직접 딕셔너리로 저장하기(이건 별수 없이 직접해야됨)
# 키워드를 아키코드로 변환 후 리스트에 저장
# 키워드 별 각 언론사에 검색하고 나오는 페이지의 제목을 가져옴
# 중복 체크후 리스트에 저장
# 각각 중복 체크 전 후로 갯수의 차이가 있는지 체크
# 다만 aving와 zdnet는 갯수의 변동이 보이나 동아일보는 갯수의 변동이 없어서 뭔가 이상함
# 네이버에서 검색하고 view에서 블로그를 선택하는게 아니라 네이버의 블로그로 들어가서 검색하는게 더..? 이러면 스크롤을 안써도 될것같은데...
# 유튭은 써야되는구만..

['%EC%BA%90%EB%85%BC%EC%BD%94%EB%A6%AC%EC%95%84', '%EC%BA%90%EB%85%BC+%EC%B9%B4%EB%A9%94%EB%9D%BC']
aving의 전체 기사 갯수:1134
0
0
0
0
0


In [ ]:
newsis

,bon
0,캐논코리아 주식회사(대표이사 최세환)가 최근 카메라 솔루션과 함께 프린팅 솔루션을 ...
1,먼저 캐논코리아만의 고품질 프린팅 솔루션은 캐논 갤러리 사진 전시회에서 확인할 수 ...
2,이번 전시회의 작품들은 모두 캐논의 라지 포맷 프린터(Large Format Pri...
3,이 외에도 이종렬 작가의 작품을 바탕으로 제작된 8분가량의 영상을 캐논의 LCOS(...
4,한편 캐논코리아는 지난 6월 코엑스에서 개최한 '제31회 서울국제사진영상기자재전(P...
...,...
262,F1.6의 밝은 단초점 렌즈가 탑재돼 있어 동영상이나 사진 촬영에서 '보케'를 자유...
263,상하좌우로 렌즈의 기울기를 바꿈으로써 초점이 맞는 범위를 변경하는 것을 '틸트 촬영...
264,"Miniature Tilt Lens는 렌즈 캡, 조리개 링, 포커스 링, 틸트 링 ..."
265,"GIZMON 창업자인 세이케 히데아키(Seike Hideaki)는 ""미니어처와 디오..."


In [ ]:
stopwords =[
  'cctv', 
  '카메라',
  '뉴스',
  '특별',
  '페이지',
  '바로',
  '가기'
]
stopwords_str = '\n'.join(stopwords)
stopwords_str
# cctv 카메라
# 핸드폰 카메라
# 몰래 카메라
# 몰레 카메라



'cctv\n카메라\n뉴스\n특별\n페이지\n바로\n가기'

In [ ]:
!pip install konlpy

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 19.4 MB 1.2 MB/s 
     |████████████████████████████████| 453 kB 57.2 MB/s 


In [ ]:
from konlpy.tag import Okt # komoran, hannanum, kkma, mecab

import os

import numpy as np
import pandas as pd

from datetime import datetime
import json
import re


from tqdm.notebook import tqdm

In [ ]:
from collections import Counter

con = Counter(total)

count = dict(sorted(con.items(), key=lambda x: x[1], reverse=True))
count

{'하다': 575,
 '있다': 245,
 '캐논': 225,
 '촬영': 167,
 '렌즈': 158,
 '되다': 103,
 '코리아': 99,
 '영상': 92,
 '제품': 71,
 '지원': 64,
 '가능하다': 61,
 '시리즈': 60,
 '사진': 58,
 '다양하다': 56,
 '최대': 55,
 '출력': 54,
 '돼다': 50,
 '시스템': 49,
 '기능': 49,
 '기술': 48,
 '출시': 46,
 '탑재': 45,
 '통해': 44,
 '갖추다': 43,
 '성능': 40,
 '대표이사': 37,
 '이번': 37,
 '프린터': 37,
 '또한': 34,
 '모델': 34,
 '방송': 34,
 '솔루션': 33,
 '선보이다': 32,
 '사용': 32,
 '디자인': 32,
 '마운트': 32,
 '늘다': 31,
 '관련': 30,
 '오다': 29,
 '시간': 28,
 '사용자': 27,
 '고속': 27,
 '무게': 26,
 '음향': 26,
 '제공': 25,
 '활용': 25,
 '유저': 25,
 '신제품': 25,
 '라인업': 24,
 '예정': 24,
 '편의': 24,
 '환가': 23,
 '받다': 23,
 '보다': 23,
 '환경': 23,
 '진행': 23,
 '주식회사': 22,
 '혁신': 22,
 '전시': 22,
 '새롭다': 22,
 '시네마': 22,
 '연속': 22,
 '확인': 21,
 '프로': 21,
 '뛰어나다': 21,
 '체험': 21,
 '고객': 21,
 '풀프레임': 21,
 '공식': 21,
 '구성': 21,
 '광각': 20,
 '미러리스': 20,
 '밝히다': 20,
 '전시회': 19,
 '고화질': 19,
 '광학': 19,
 '작업': 19,
 '망원': 19,
 '화각': 19,
 '가격': 19,
 '포토': 19,
 '자동': 18,
 '한편': 18,
 '초점': 18,
 '기록': 18,
 '기존': 18,
 '콤팩트': 18,
 '처리

In [ ]:
a = {
    'a':1,
     'b':2,
     'c':3
}


['a', 'b', 'c']

In [ ]:
# 1. 한글 및 공백을 제외한 문자 모두 제거.
trend_text = re.sub("[^가-힣\\s]", "", trend)
trend_text

''

In [ ]:
# 2. okt 객체를 활용해서 형태소 토큰화 + 품사 태깅 에다가 통일화까지
okt = Okt() # 형태소 분석기
word_trend = okt.pos(trend_text, stem=True) # stem = 어감 추출
word_trend

[('캐논', 'Noun'),
 ('코리아', 'Noun'),
 ('주식회사', 'Noun'),
 ('대표이사', 'Noun'),
 ('최', 'Noun'),
 ('세', 'Modifier'),
 ('환가', 'Noun'),
 ('최근', 'Noun'),
 ('카메라', 'Noun'),
 ('솔루션', 'Noun'),
 ('과', 'Josa'),
 ('함께', 'Adverb'),
 ('프린팅', 'Noun'),
 ('솔루션', 'Noun'),
 ('을', 'Josa'),
 ('소개', 'Noun'),
 ('하다', 'Verb'),
 ('입력', 'Noun'),
 ('에서', 'Josa'),
 ('출력', 'Noun'),
 ('까지', 'Josa'),
 ('모두', 'Noun'),
 ('를', 'Josa'),
 ('제공', 'Noun'),
 ('하다', 'Verb'),
 ('토털', 'Noun'),
 ('이미', 'Adverb'),
 ('징', 'Noun'),
 ('솔루션', 'Noun'),
 ('메시지', 'Noun'),
 ('를', 'Josa'),
 ('알리', 'Noun'),
 ('며', 'Josa'),
 ('적극', 'Noun'),
 ('적', 'Suffix'),
 ('인', 'Josa'),
 ('행보', 'Noun'),
 ('를', 'Josa'),
 ('이', 'Determiner'),
 ('어가', 'Noun'),
 ('고', 'Josa'),
 ('있다', 'Adjective')]

In [ ]:
# token의 길이가 2개 이상이고 token이 stopwords안에 없을 때만 출력해보자
for token, pom in word_trend:
  if len(token) >= 2 and not token in stopwords:
    print(token, pom)

캐논 Noun
코리아 Noun
주식회사 Noun
대표이사 Noun
환가 Noun
최근 Noun
솔루션 Noun
함께 Adverb
프린팅 Noun
솔루션 Noun
소개 Noun
하다 Verb
입력 Noun
에서 Josa
출력 Noun
까지 Josa
모두 Noun
제공 Noun
하다 Verb
토털 Noun
이미 Adverb
솔루션 Noun
메시지 Noun
알리 Noun
적극 Noun
행보 Noun
어가 Noun
있다 Adjective


In [ ]:
for x in news:
    print(preprocessing(x))

['캐논', '코리아', '주식회사', '대표이사', '환가', '최근', '솔루션', '프린팅', '솔루션', '소개', '하다', '입력', '출력', '모두', '제공', '하다', '토털', '솔루션', '메시지', '알리', '적극', '행보', '어가', '있다']
['먼저', '캐논', '코리아', '품질', '프린팅', '솔루션', '캐논', '갤러리', '사진', '전시회', '확인', '하다', '있다', '오다', '개최', '하다', '종렬', '작가', '사진', '전시회', '노숙', '캐논', '혁신', '프린팅', '기술', '만들다', '내다', '다양하다', '작품', '감상', '하다', '있다', '이번', '사진전', '캐논', '아웃', '솔루션', '통해', '입력', '출력', '캐논', '혁신', '기술', '만들다', '내다', '결과물', '직접', '경험', '하다', '있다', '의의', '있다']
['이번', '전시회', '작품', '모두', '캐논', '라지', '포맷', '프린터', '이하', '제품', '출력', '하다', '최대', '인치', '사이즈', '인쇄', '하다', '있다', '상업', '용지', '종류', '자동', '감지', '하다', '다양하다', '사이즈', '출력', '손쉽다', '출력', '하다', '있다', '충실하다', '재현', '고화질', '실현', '하다', '루시아', '프로', '잉크', '채택', '개발', '통합', '프린트', '헤드', '적용', '하다', '뛰어나다', '발색', '더욱', '선명하다', '화상', '구현', '하다']
['종렬', '작가', '작품', '바탕', '제작', '되다', '가량', '영상', '캐논', '방식', '리얼', '레이저', '프로젝터', '통해', '상영', '하다', '생동감', '넘치다', '촬영', '현장', '모습', '관람', '하다', '있다']
['한편', '캐논', '코리아', '지난', '코엑스', 

In [ ]:
es = {
    '언론사 이름1' : [[1,2,3],[4,5,6]],
    '언론사 이름2' : [['본문1'],['본문2']]
}

ns=pd.DataFrame(es)
ns

,언론사 이름1,언론사 이름2
0,"[1, 2, 3]",[본문1]
1,"[4, 5, 6]",[본문2]


In [ ]:
newsis

,bon
0,캐논코리아 주식회사(대표이사 최세환)가 최근 카메라 솔루션과 함께 프린팅 솔루션을 ...
1,먼저 캐논코리아만의 고품질 프린팅 솔루션은 캐논 갤러리 사진 전시회에서 확인할 수 ...
2,이번 전시회의 작품들은 모두 캐논의 라지 포맷 프린터(Large Format Pri...
3,이 외에도 이종렬 작가의 작품을 바탕으로 제작된 8분가량의 영상을 캐논의 LCOS(...
4,한편 캐논코리아는 지난 6월 코엑스에서 개최한 '제31회 서울국제사진영상기자재전(P...
...,...
486,"Miniature Tilt Lens는 렌즈 캡, 조리개 링, 포커스 링, 틸트 링 ..."
487,
488,"GIZMON 창업자인 세이케 히데아키(Seike Hideaki)는 ""미니어처와 디오..."
489,


In [ ]:
url = 'https://kr.aving.net/news/articleView.html?idxno=1728142'
resp = requests.get(url)
soup = BeautifulSoup(resp.content, 'lxml')
tag = soup.select('div.article-body p')
t = []
for ve in tag:
  if ve != '':
    t.append(ve.text)

t

['',
 "캐논코리아 주식회사(대표이사 최세환)가 광각 단초점 렌즈 'RF24mm F1.8 MACRO IS STM'과 광각 줌 렌즈 'RF15-30mm F4.5-6.3 IS STM' 2종을 공개했다.",
 '',
 '새롭게 선보이는 렌즈 2종은 키트에 포함되는 렌즈를 넘어서서 보다 선명한 화질을 표현하거나, 키트 렌즈와 병용해 다채로운 사진 및 영상을 촬영할 수 있도록 지원해 풀프레임 및 APS-C의 엔트리부터 하이 아마추어 유저, 동영상 SNS 유저 등 폭넓은 고객들에게 매력적인 제품이다. 캐논코리아는 고성능에 더해 더욱 작고 가벼워진 렌즈를 출시함으로써 렌즈 라인업 및 기동성을 확대할 계획이다.',
 '',
 "탁월한 배경 흐림 효과를 지원하는 광각 단초점 렌즈 'RF24mm F1.8 MACRO IS STM'은 소형·경량화를 실현한 제품이다. 약 63.1mm에 불과한 전체 길이와 약 270g의 무게로 미러리스 카메라의 기동력과 휴대성을 살린 설계가 특징이다. F1.8의 밝은 조리개로 키트 줌 렌즈보다 약 2스톱 더 빠른 셔터 스피드 촬영을 지원해 보다 깨끗한 배경 흐림 효과를 구현한다. 또한, 0.14m의 짧은 초점거리로 최대 촬영 배율 0.5배의 하프 매크로 촬영도 지원한다. RF 단초점 렌즈 시리즈는 RF24mm F1.8 MACRO IS STM의 출시로 고객들에게 광각부터 망원까지 빠짐없는 화각을 커버하며, 줌 렌즈에는 없는 선명함과 고화질 성능을 누릴 수 있게 됐다.",
 '',
 'RF24mm F1.8 MACRO IS STM은 바디 내 5축 손떨림 보정 기능을 갖춘 EOS R3 및 EOS R5와 결합 시 최대 6.5스톱의 강력한 통합 제어 IS 성능도 갖췄다. 렌즈 자체만으로도 최대 5스톱의 손떨림 보정을 지원해 조도가 낮은 곳에서도 핸드 헬드 촬영 시 선명한 이미지를 구현한다. 실내에서도 촬영하기 쉬운 넓은 화각과 소형, 경량 사이즈, 그리고 뛰어난 손떨림 보정 기능과 자연스러운 동영상 AF 기능을 갖춰 동영상 촬영에도 적합한 렌즈다.',
 ''

In [ ]:
for i in range(5):
  for j in range(3):
    print(i,j)

0 0
0 1
0 2
1 0
1 1
1 2
2 0
2 1
2 2
3 0
3 1
3 2
4 0
4 1
4 2


실시간 보다는 미리 가져온 데이터로 작업
토픽 모델
DB로 데이터를 미리 저장
키워드를 토큰화 해서 가격과 관련된 사전을 만들고,








제품을 정해서 DB를 쌓고 
라이브러리 캘린더 이용



In [ ]:
t = {
    1 : {
        2:[3,4]
    },
    5 : {
        2 : [7,8]
    }
}

nf = pd.DataFrame(t)
nf

,1,5
2,"[3, 4]","[7, 8]"


나왔던 이슈
뉴스의 본내용 크롤링 하기
 - 검색하고 나서 뉴스의 본내용을 가져올 때 태그 및 class가 전부 다름
 - 포기하고 검색했을 때에 나오는 제목,요약내용,날짜, 언론사를 크롤링 하기로 함

view 크롤링 하기
 - view가 블로그와 카페의 합집합이라는 것을 늦게 깨달음
 - 카페는 로그인의 문제가 아닌 로그인 후 가입의 문제이므로 포기
 - 블로그는 크롤링이 가능하나 iframe의 유무가 존재함
 - 스크롤기능을 활용할 수 있는 방안이 있는지

slrclub을 활용할지


In [ ]:
replace('\n','') = 개행문자? 를 지우기 위해 활용

In [ ]:
근데 컬럼에 일괄적용 하고 싶을 때,
ex) df 라는 데이터프레임이 있다고 하면
벨류 안에 개행문자가 포함되어 있을 때,
df.review.str.replace('\n', '')
정규식도 비슷
df.review.str.정규식